In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, get_scorer
from sklearn.linear_model import Lasso, Ridge, LassoCV, LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import KFold, RepeatedKFold, GridSearchCV, cross_validate, train_test_split

# LassoCV 사용해보기
[출처](https://towardsdatascience.com/hyperparameter-tuning-in-lasso-and-ridge-regressions-70a4b158ae6d)

In [2]:
d1221 = pd.read_feather('d1221_10thr.ftr', use_threads = True, columns = None)

## mk_cd 함수

In [3]:
## 함수
def mk_cd(x):
    if type(x) == str:
        if x.isalpha():
            return x
        else:
            if len(x) == 6:
                return x
            else:
                return "0"*(6-len(x))+x
    elif type(x) == int:
        x = str(x)
        return "0"*(6-len(x))+x

# 슬라이싱 준비

In [4]:
len_a2012 = sum((20120000<d1221['date'])&(d1221['date']<20130000)) # 107935
len_a2013 = sum((20130000<d1221['date'])&(d1221['date']<20140000))
len_a2014 = sum((20140000<d1221['date'])&(d1221['date']<20150000))
len_a2015 = sum((20150000<d1221['date'])&(d1221['date']<20160000))
len_a2016 = sum((20160000<d1221['date'])&(d1221['date']<20170000))
len_a2017 = sum((20170000<d1221['date'])&(d1221['date']<20180000))
len_a2018 = sum((20180000<d1221['date'])&(d1221['date']<20190000))
len_a2019 = sum((20190000<d1221['date'])&(d1221['date']<20200000))
len_a2020 = sum((20200000<d1221['date'])&(d1221['date']<20210000))
len_a2021 = sum((20210000<d1221['date'])&(d1221['date']<20220000))

In [5]:
len1215 = len_a2012+len_a2013+len_a2014+len_a2015 # y:2015
len1316 = len1215+len_a2016 # y:2016
len1417 = len1316+len_a2017 # y:2017
len1518 = len1417+len_a2018 # y:2018
len1619 = len1518+len_a2019 # y:2019
len1720 = len1619+len_a2020 # y:2020
len1821 = len1720+len_a2021 # y:2021

## [0], [1]은 train 슬라이싱 인덱스, [2]는 test인덱스로 구성

In [6]:
slicing_dic = {'1215':[0, len1215, len_a2015],
               '1316':[len_a2012, len1316, len_a2016],
               '1417':[len_a2012+len_a2013, len1417, len_a2017],
               '1518':[len_a2012+len_a2013+len_a2014, len1518, len_a2018],
               '1619':[len_a2012+len_a2013+len_a2014+len_a2015, len1619, len_a2019],
               '1720':[len_a2012+len_a2013+len_a2014+len_a2015+len_a2016,len1720, len_a2020],
               '1821':[len_a2012+len_a2013+len_a2014+len_a2015+len_a2016+len_a2017,len1821, len_a2021]}

# category to object

In [7]:
d1221['code'] = list(map(mk_cd, d1221['code'])) # mk_cd함수 적용시켜야 ftr만들 때 에러 안남.

In [8]:
d1221.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1439871 entries, 0 to 1439870
Columns: 823 entries, date to NextChange
dtypes: float64(821), int64(1), object(1)
memory usage: 8.8+ GB


# reg modeling

In [10]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
lasso_alphas = np.linspace(0, 0.2, 21)

In [ ]:
results = []
def my_regressor():
    X = d1221.iloc[:,2:-1] # 1221 데이터
    y = d1221.iloc[:,-1]
    
    from sklearn.model_selection import ShuffleSplit
    from sklearn.model_selection import cross_val_score
    
    from sklearn.dummy import DummyRegressor
    from sklearn.metrics import mean_squared_error as mse # 모델 평가 지표 scoring (mse)
    from sklearn.metrics import r2_score as r2
    
    # seed 고정
    user_seed = 0
    random.seed(user_seed)
    
    lasso = LassoCV(alphas=lasso_alphas, cv=cv, n_jobs=64)
    my_model_list = ['lasso']
    
    i = 0
    

    ######################################################################### fitting
    for model_nm in tqdm(my_model_list):
        cntcnt = 0
        # cv
        for slicer in slicing_dic:
            tmpX = X[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True)
            tmpy = y[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True)
#             tmp_all = d1221[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # 전체데이터

            X_train = tmpX[:-slicing_dic[slicer][2]]
            X_test = tmpX[-slicing_dic[slicer][2]:]
            y_train = tmpy[:-slicing_dic[slicer][2]]
            y_test = tmpy[-slicing_dic[slicer][2]:]
            
            # train끼리 idx 같아야 하므로
            len_train = slicing_dic[slicer][1]-(slicing_dic[slicer][2]+slicing_dic[slicer][0]) # 혹은 tmp_train.shape[0]
            len(list(range(len_train)))
            train_idx = list(range(len_train))
            random.shuffle(train_idx) # 자동으로 덮어쓰기

            len_train_to_test = slicing_dic[slicer][1]-slicing_dic[slicer][0]
            test_idx = list(range(len_train,len_train_to_test)) # +1 안해줘도 되는 거 위 cell에서 확인
            len(test_idx)
            random.shuffle(test_idx) # 자동으로 덮어쓰기
            
            X_train = X_train.loc[train_idx]
            y_train = y_train.loc[train_idx]
            X_test = X_test.loc[test_idx]
            y_test = y_test.loc[test_idx]
            ###########################################
            y_train = np.array(y_train).reshape(-1,1) # 1열짜리로 만드는 것
            y_test = np.array(y_test).reshape(-1,1)

            ######################################################## cv를 위한 데이터 slicing 작업 끝
            
            # fitting은 cv 안에 넣기
            cntcnt += 1
            print(f'$$$$$---{cntcnt}')
            model_ = eval(model_nm).fit(X_train, y_train)
            y_pred_test = model_.predict(X_test) # 예측
            
            mse_score = mse(y_test, y_pred_test)
            r2_score = r2(y_test, y_pred_test)
            print('alpha: %.2f' % lasso.alpha_)
            results.append([i, f'{model_nm}', mse_score, r2_score])
            i+=1
        ##############################################################################
        print(results[-7:])
    return results#score_dic#[0]

In [ ]:
# 최종
import time
start = time.time()
_ = my_regressor()
results
time.time()-start

# random search CV

## library

In [9]:
from sklearn.model_selection import RandomizedSearchCV
# 출처: https://kimmaadata.tistory.com/40 [Data Scientist Kimmaa's log:티스토리]

## 파이프라인을 이용한 모델학습
--> 안함

In [10]:
! pip install category_encoders

Defaulting to user installation because normal site-packages is not writeable


In [11]:
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# pipe = make_pipeline(
#     OneHotEncoder(use_cat_names=True)
#     , SimpleImputer()
#     , StandardScaler()
#     , SelectKBest(f_regression)
#     , Ridge())

In [12]:
results = []
def my_regressor():
    X = d1221.iloc[:,2:-1] # 1221 데이터
    y = d1221.iloc[:,-1]
    
    from sklearn.model_selection import ShuffleSplit
    from sklearn.model_selection import cross_val_score
    
    from sklearn.dummy import DummyRegressor
    from sklearn.metrics import mean_squared_error as mse # 모델 평가 지표 scoring (mse)
    from sklearn.metrics import r2_score as r2
    from sklearn.linear_model import Lasso, Ridge, LassoCV, LinearRegression
    
    # seed 고정
    user_seed = 0
    random.seed(user_seed)
    
#     lasso = LassoCV(alphas=lasso_alphas, cv=cv, n_jobs=64)
#     Ridge = Ridge()
    
    # 1. pipe라인을 이용하여 모델 학습(예시는 릿지회귀)
    pipe = make_pipeline(
                        OneHotEncoder(use_cat_names=True)
                        , SimpleImputer()
                        , StandardScaler()
                        , SelectKBest(f_regression)
                        , Ridge())
    
    my_model_list = ['Ridge']
    
    i = 0

    ######################################################################### fitting
    for model_nm in tqdm(my_model_list):
        print(f'<<<---{model_nm} start--->>>')
        print(datetime.now().strftime('%H:%M:%S'))
        cntcnt = 0
        
        # cv
        for slicer in slicing_dic:
            tmpX = X[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True)
            tmpy = y[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True)
#             tmp_all = d1221[slicing_dic[slicer][0]:slicing_dic[slicer][1]].reset_index(drop = True) # 전체데이터

            X_train = tmpX[:-slicing_dic[slicer][2]]
            X_test = tmpX[-slicing_dic[slicer][2]:]
            y_train = tmpy[:-slicing_dic[slicer][2]]
            y_test = tmpy[-slicing_dic[slicer][2]:]
            
            # train끼리 idx 같아야 하므로
            len_train = slicing_dic[slicer][1]-(slicing_dic[slicer][2]+slicing_dic[slicer][0]) # 혹은 tmp_train.shape[0]
            len(list(range(len_train)))
            train_idx = list(range(len_train))
            random.shuffle(train_idx) # 자동으로 덮어쓰기

            len_train_to_test = slicing_dic[slicer][1]-slicing_dic[slicer][0]
            test_idx = list(range(len_train,len_train_to_test)) # +1 안해줘도 되는 거 위 cell에서 확인
            len(test_idx)
            random.shuffle(test_idx) # 자동으로 덮어쓰기
            
            X_train = X_train.loc[train_idx]
            y_train = y_train.loc[train_idx]
            X_test = X_test.loc[test_idx]
            y_test = y_test.loc[test_idx]
            ###########################################
            y_train = np.array(y_train).reshape(-1,1) # 1열짜리로 만드는 것
            y_test = np.array(y_test).reshape(-1,1)

            ######################################################## cv를 위한 데이터 slicing 작업 끝
            
            # 2. 튜닝할 하이퍼파라미터의 범위를 지정
            dists = {
                    'simpleimputer__strategy': ['mean', 'median'], 
                    'selectkbest__k': range(1, len(X_train.columns)+1), 
                    'ridge__alpha': [0.1, 1, 10]}
            
            # 3. Randomized Search CV 진행
            clf = RandomizedSearchCV(
                                    pipe,                       #파이프라인으로 학습된 모델 
                                    param_distributions=dists,  #하이퍼파라미터 튜닝 값으로 진행한다
                                    n_iter=50,                  # 반복 횟수
                                    cv=3,                       # 교차검증 횟수 (==> n_iter * cv 의 숫자만큼 진행됨)
                                    scoring='neg_mean_absolute_error', # MAE
                                    verbose=1,                  # 훈련 중지여부를 화면에 출력 (1= progress bar/2= one line per epoch)
                                    n_jobs=12)
            
            
            # fitting은 cv 안에 넣기
            cntcnt += 1
            print(f'$$$$$---{cntcnt}')
#             model_ = eval(model_nm).fit(X_train, y_train)
            
            
            
            # Randomized Search CV로 train data 학습진행
            clf.fit(X_train, y_train)
            print('최적 하이퍼파라미터: ', clf.best_params_)
            print('MAE: ', -clf.best_score_)
            
            # rank_test_score: 테스트 순위
            # mean_score_time: 예측에 걸리는 시간
            pd.DataFrame(clf.cv_results_).sort_values(by='rank_test_score').T
            
#             y_pred_test = model_.predict(X_test) # 예측
            
#             mse_score = mse(y_test, y_pred_test)
#             r2_score = r2(y_test, y_pred_test)
#             print('alpha: %.2f' % lasso.alpha_)
#             results.append([i, f'{model_nm}', mse_score, r2_score])
            i+=1
        ##############################################################################
#         print(results[-7:])
    return results#score_dic#[0]

In [13]:
import time
start = time.time()
_ = my_regressor()
results
time.time()-start

  0%|                                                     | 0/1 [00:00<?, ?it/s]

<<<---Ridge start--->>>
18:03:58
$$$$$---1
Fitting 3 folds for each of 50 candidates, totalling 150 fits
최적 하이퍼파라미터:  {'simpleimputer__strategy': 'median', 'selectkbest__k': 483, 'ridge__alpha': 10}
MAE:  2.8160524190453757
$$$$$---2
Fitting 3 folds for each of 50 candidates, totalling 150 fits
최적 하이퍼파라미터:  {'simpleimputer__strategy': 'mean', 'selectkbest__k': 663, 'ridge__alpha': 10}
MAE:  2.8512358276923133
$$$$$---3
Fitting 3 folds for each of 50 candidates, totalling 150 fits


/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

...

/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

In [14]:
results

[]

In [ ]:
import time
start = time.time()
_ = my_regressor()
results
time.time()-start

  0%|                                                     | 0/1 [00:00<?, ?it/s]

<<<---Ridge start--->>>
17:04:20
$$$$$---1
Fitting 3 folds for each of 50 candidates, totalling 150 fits
최적 하이퍼파라미터:  {'simpleimputer__strategy': 'mean', 'selectkbest__k': 486, 'ridge__alpha': 10}
MAE:  2.8160426460191803
$$$$$---2
Fitting 3 folds for each of 50 candidates, totalling 150 fits


/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/package/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

최적 하이퍼파라미터:  {'simpleimputer__strategy': 'median', 'selectkbest__k': 663, 'ridge__alpha': 10}
MAE:  2.8512358276923133
$$$$$---3
Fitting 3 folds for each of 50 candidates, totalling 150 fits
